# 2. Properties of CO$_2$

## 2.1. Pure CO$_2$ enthalpy and Joule-Thomson coefficient

In [ ]:
import numpy as np
import xarray as xr
from dartsflash.pyflash import PyFlash
from dartsflash.mixtures import Mixture
from dartsflash.libflash import CubicEoS


R = 8.3144772  # gas constant

# Define mixture and initialize Flash object
mix = Mixture(components=["CO2"])
f = PyFlash(mixture=mix)

# Add CubicEoS object
f.add_eos("CEOS", CubicEoS(mix.comp_data, CubicEoS.PR))
ceos = f.eos["CEOS"]

# Plot enthalpy diagram
state_spec = {"temperature": np.arange(273, 423, 1),
              "pressure": np.arange(1, 200, 1),
             }
compositions = {"CO2": 1.}
properties = {"H": ceos.H_PT,
              "JT": ceos.JT,
             }

pt_props = f.evaluate_properties_1p(state_spec=state_spec, compositions=compositions,
                                    properties_to_evaluate=properties, mole_fractions=True)

## 2.1a. Enthalpy

In [ ]:
# Plot PT diagram of CO2 enthalpy
from dartsflash.plot import PlotEoS
pt = PlotEoS.surf(f, pt_props, x_var="temperature", y_var="pressure", prop_names=["H"], composition=[1.])
pt.draw_point(X=304.1, Y=73.75, colours='red')  # Critical point

pt.add_attributes(ax_labels=["temperature, K", "pressure, bar"])

## 2.1b. Joule-Thomson coefficient

In [ ]:
# Get reference data from NIST
from ref_data import get_nist_data

temperatures = np.array([0., 30.]) + 273.15
ref_p, ref_JT = [[] for i, temp in enumerate(temperatures)], [[] for i, temp in enumerate(temperatures)]
for i, t in enumerate(temperatures):
    nist_data = get_nist_data(component="CO2", temperature=t, 
                              pressure=np.arange(25, 301, 25), include_phase_boundary=True)

    ref_p[i] = nist_data["pressure"]
    ref_JT[i] = nist_data["jt"]

In [ ]:
# Joule-Thomson coefficient CO2
from dartsflash.plot import PlotEoS
jt = PlotEoS.plot(f, pt_props, composition=[1.], x_var="pressure", prop_names=["JT"],
                  state={"temperature": temperatures}, labels=["T = {:.0f} K".format(t) for t in temperatures])

jt.draw_refdata(xref=ref_p, yref=ref_JT)
jt.add_attributes(ax_labels=["pressure, bar", "JT, K/bar"], legend=True)

### <font color='Red'> Excersise: plot cooling curves for T = 60 C and 90 C with the same pressure drop

## 2.2. PH-diagram of CO$_2$

In [ ]:
#  PH diagram
from dartsflash.libflash import PHFlash
Trange = [260., 350.]
Prange = [10., 100.]

f.flash_params.phflash_Htol = 1e-3
f.flash_params.phflash_Ttol = 1e-8
f.f = PHFlash(f.flash_params)
f.np_max = 2

# Evaluate
Hrange = [f.eos["CEOS"].H_PT(Prange[1], Ti, [1.]) * R for Ti in Trange]
state_spec = {"pressure": np.linspace(Prange[0], Prange[1], 200),
              "enthalpy": np.linspace(Hrange[0], Hrange[1], 100),
              }

flash_results = f.evaluate_flash_1c(state_spec=state_spec)

# Plot PH diagram
from dartsflash.plot import PlotFlash
PlotFlash.ph(f, flash_results, composition=[1.], plot_temperature=True)

## 2.3. Isenthalpic cooling

In [ ]:
# Plot PT diagram of CO2 enthalpy
from dartsflash.plot import PlotEoS
pt = PlotEoS.surf(f, pt_props, x_var="temperature", y_var="pressure", prop_names=["H"], composition=[1.])
pt.draw_point(X=304.1, Y=73.75, colours='red')  # Critical point

pt.add_attributes(ax_labels=["temperature, K", "pressure, bar"])

R = 8.3144
enthalpies = [ceos.H_PT(p=100., T=temp, n=[1.]) * R for temp in [273, 303, 333]]

pressures = [flash_results.pressure.values for h in enthalpies]
temps = [flash_results.sel(enthalpy=h, method='nearest').temp.values for h in enthalpies]

for i in range(3):
    pt.draw_line(temps[i], pressures[i])
# """"""